In [2]:
import os
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [3]:
def compute_RS_rolling(series, window_size=30):
    n = len(series)
    RS = []
    dates = []
    
    for start in range(0, n - window_size + 1):
        end = start + window_size
        segment = series[start:end]
        mean = segment.mean()
        cumulative_deviation = np.cumsum(segment - mean)
        R = np.max(cumulative_deviation) - np.min(cumulative_deviation)
        S = segment.std()
        if S != 0:
            RS.append(R / S)
            dates.append(series.index[end-1])
    
    return dates, RS

def hurst_exponent_rolling(series, window_size=30):
    dates, RS = compute_RS_rolling(series, window_size)
    if not RS:
        return pd.DataFrame(columns=['Date', 'Hurst Exponent'])
    

    hurst_values = []
    for rs in RS:
        hurst_values.append(np.log(rs)/np.log(window_size))
    
    results_df = pd.DataFrame({'Date': dates, 'Hurst Exponent': hurst_values})
    return results_df

In [4]:
file_path= '/Users/jinnia/Desktop/data/sp6m_6.csv'
df = pd.read_csv(file_path, parse_dates=['idt'])
df.set_index('idt', inplace=True)

timeseries = df['mu']
results_df = hurst_exponent_rolling(timeseries, 30)
print(results_df)


          Date  Hurst Exponent
0   2008-03-31        0.736254
1   2008-04-15        0.744547
2   2008-04-30        0.750150
3   2008-05-15        0.754461
4   2008-05-30        0.756937
..         ...             ...
651 2023-12-13        0.697537
652 2023-12-20        0.716123
653 2023-12-27        0.718466
654 2024-01-03        0.723428
655 2024-01-10        0.729150

[656 rows x 2 columns]


In [5]:
hurst_avg = results_df['Hurst Exponent'].mean()
results_df['index'] = results_df['Hurst Exponent'].apply(lambda x: -1 if x > hurst_avg else 1)


In [6]:
sp_df = pd.read_csv('/Users/jinnia/Desktop/data/sp.csv', parse_dates=['Date'])
sp_df.set_index('Date', inplace=True)
sp_selected = sp_df[['Open','High','Low','Close']]
merged_df = results_df.join(sp_selected, on='Date', how='left')
merged_df.dropna(subset=['Open'], inplace=True)
merged_df = merged_df.drop('Hurst Exponent', axis=1)

print(merged_df)


          Date  index         Open         High          Low        Close
0   2008-03-31     -1  1315.920044  1328.520020  1312.810059  1322.699951
1   2008-04-15     -1  1331.719971  1337.719971  1324.349976  1334.430054
2   2008-04-30     -1  1391.219971  1404.569946  1384.250000  1385.589966
3   2008-05-15     -1  1408.359985  1424.400024  1406.869995  1423.569946
4   2008-05-30     -1  1398.359985  1404.459961  1398.079956  1400.380005
..         ...    ...          ...          ...          ...          ...
597 2022-11-30     -1  3957.179932  4080.110107  3938.580078  4080.110107
598 2022-12-07     -1  3933.280029  3957.570068  3922.679932  3933.919922
599 2022-12-14     -1  4015.540039  4053.760010  3965.649902  3995.320068
600 2022-12-21     -1  3839.489990  3889.820068  3839.489990  3878.439941
601 2022-12-28     -1  3829.560059  3848.320068  3780.780029  3783.219971

[601 rows x 6 columns]


In [7]:
cols = [col for col in merged_df.columns if col != 'index']
cols.append('index')
merged_df = merged_df[cols]
merged_df

,Date,Open,High,Low,Close,index
0,2008-03-31,1315.920044,1328.520020,1312.810059,1322.699951,-1
1,2008-04-15,1331.719971,1337.719971,1324.349976,1334.430054,-1
2,2008-04-30,1391.219971,1404.569946,1384.250000,1385.589966,-1
3,2008-05-15,1408.359985,1424.400024,1406.869995,1423.569946,-1
4,2008-05-30,1398.359985,1404.459961,1398.079956,1400.380005,-1
...,...,...,...,...,...,...
597,2022-11-30,3957.179932,4080.110107,3938.580078,4080.110107,-1
598,2022-12-07,3933.280029,3957.570068,3922.679932,3933.919922,-1
599,2022-12-14,4015.540039,4053.760010,3965.649902,3995.320068,-1
600,2022-12-21,3839.489990,3889.820068,3839.489990,3878.439941,-1


In [8]:
merged_df.to_csv('/Users/jinnia/Desktop/backtest/raw data/backtesting_data.csv', index=False)

In [10]:
%run /Users/jinnia/Desktop/backtest/cringe.py


root - INFO - 启动资金: 1000000.00
root - INFO - 期末价值: 1196743.47
root - INFO - Annual Return: 0.03558292413943511
root - INFO - Sharp Ratio: 0.5081650598749003
root - INFO - Max DrawDown: 83.91521081804929%


2008-04-15 23:59:59.999989,卖出:backtesting_data
价格:1331.72,                发起：sell,                成本: -503390.15,                手续费10.07
2010-10-15 23:59:59.999989,买入:backtesting_data
价格:1177.47,                发起：close,                成本:-503390.15,                手续费:8.90
2010-10-15 23:59:59.999989,买入:backtesting_data
价格:1177.47,                发起：buy,                成本:552233.42,                手续费:11.04
2010-10-15 23:59:59.999989,策略收益：
毛收益 58306.50, 净收益 58287.53
2013-01-24 23:59:59.999989,卖出:backtesting_data
价格:1494.81,                发起：close,                成本: 552233.42,                手续费14.02
2013-01-24 23:59:59.999989,卖出:backtesting_data
价格:1494.81,                发起：sell,                成本: -606892.88,                手续费12.14
2013-01-24 23:59:59.999989,策略收益：
毛收益 148832.50, 净收益 148807.44
2013-02-07 23:59:59.999989,买入:backtesting_data
价格:1512.12,                发起：close,                成本:-606892.88,                手续费:12.28
2013-02-07 23:59:59.999989,买入:backtesting_data
价格:1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

: 

H is typically above 0.5 in times of smooth returns and low volatility, indicating positive expected autocorrelation.         
In times of nervousness, market expectations either revert to a random walk (H = 0.5) or lead to negative ex-ante momentum.                     
https://doi.org/10.1016/j.frl.2019.101347